1-ый вариант арифметического кодирования с использованием десятичных чисел

In [4]:
from typing import Dict, List  #подключаем модуль typing для аннотации типов
from decimal import Decimal, getcontext

getcontext().prec = 300  #количество занков после запятой

In [4]:
def countProbability(text: str) -> Dict[str, Decimal]:  #функция для подсчета частоты встречаемых символов
    textSize = len(text)  #длина сообщения, которое нужно закодировать
    probability = {}  #словарь для подсчета частоты встречаемых символов (ключ - символ, значение - количество данного символа в тексте)
    for char in text:  #цикл, который итерируется по сообщению
        if char in probability:  #если ключ char есть в словаре, то +1 к его значению
            probability[char] += 1
        else:
            probability[char] = 1  #иначе значение ключа = 1
    probability = {key: Decimal(value) / Decimal(textSize) for key, value in probability.items()}  #насчитываем частоту появления каждого уникального символа
    return probability  #возвращаем итоговый словарь

In [ ]:
def buildSegmentDict(probability: Dict[str, Decimal]) -> Dict[str, List[Decimal]]:  #функция для составления "рабочих" полуинтервалов для каждого символа, где длина полуинтервала = частоте появления символа
    segmentDict = {}  #словарь для составления полуинтервалов (ключ - символ, значение - список, где 0-ой элемент - левая граница, а 1-ый элемент - правая граница
    left = Decimal(0) #левая граница - 0
    for char in probability.keys():  #итерируемся по уникальным символам и составлям "рабочий" полуинтервал для каждого символа
        segmentDict[char] = [left, left + probability[char]]
        left = left + probability[char]
    return segmentDict  #возвращаем итоговый словарь

In [ ]:
def encode(text: str, segmentDict: Dict[str, List[Decimal]]) -> Decimal:  #функция для кодирования текста в число
    l, r = 0, 1  #левая граница полуинтервала - 0, правая - 1
    for char in text:  #итерируемся по каждому символу текста и пересчитываем левую и правую границу полуинтервала
        newRight = l + (r - l) * segmentDict[char][1]
        newLeft = l + (r - l) * segmentDict[char][0]
        l = newLeft
        r = newRight
    return (l + r) / 2  #в результате наш текст - любое число из полуинтервала [l, r), для лучшей точности возьмем середину

In [ ]:
def decode(segmentDict: Dict[str, List[Decimal]], code: Decimal) -> str:  #функция для декодирования числа в текст
    ans = ""  #наш текст, который должен получиться, лежит в переменной ans
    while True:   #запускаем бесконечный цикл, из которого мы выйдем только тогда, когда встретим остановочный символ
        for char, segment in segmentDict.items():  #итерируемся по словарю с "рабочими" полуинтервалами
            if char == "ඞ": #если встретили, то возвращаем ответ
                return ans
            if segment[0] <= code < segment[1]:  # если код(число - которое получили в процессе кодирования текста) лежит в полуинтервале [l, r), то это нужный нам символ
                ans += char  #кладем символ в ответ
                code = (code - segment[0]) / (segment[1] - segment[0])  #пересчитываем код для дальнейшего декодирования
                break

In [ ]:
def solution():
    with open("in.txt", 'r', encoding='utf-8') as file:
        textToEncode = file.read()
    probabilityDict = countProbability(textToEncode)  #словарь частот встречаемых символов
    segmentDict = buildSegmentDict(probabilityDict)  #словарь с "рабочими" полуинтервалами
    encodedResult = encode(textToEncode, segmentDict)  #закодированный текст
    print("Encoded text:", encodedResult)  #вывод числа
    decodedResult = decode(segmentDict, encodedResult)  #декодированное число(текст)
    print(decodedResult)  #вывод текста

In [5]:
solution()

Encoded text: 0.0000352534345940276328141935271864563592249163800409361591493566223981870793389728694301235268970069437968993894620592724794543932527947358375229899501523485818164825225646176584044404599896875104735130000256417182573454178175861153858029212871688217728168968495953168223529008007255150493911240063503471
Все символы такого алфавита пронумерованы от 0 до 255, а каждому номеру соответствует 8-разрядный двоичный код от 00000000 до 11111111.
Этот код является порядковым номером символа в двоичной системе счисления.


2 Вариант Арифметического кодирования (более надежный) с использованием рациональных чисел

In [7]:
from typing import Dict, List  # подключаем модуль typing для аннотаций типов
from fractions import Fraction  # подключаем модуль fractions для работы с рациональными числами

In [ ]:
def countProbability(text: str) -> Dict[str, Fraction]:  # Функция для подсчета частоты встречаемых символов
    textSize = len(text)  # длина сообщения, которое нужно закодировать
    probability = {}  # словарь для подсчета частоты символов (ключ - символ, значение - количество данного символа в тексте)
    for char in text:  # цикл, который итерируется по сообщению
        if char in probability:  # если ключ char есть в словаре, то +1 к его значению
            probability[char] += 1
        else:
            probability[char] = 1  # иначе значение ключа = 1
    probability = {key: Fraction(value, textSize) for key, value in probability.items()}  # насчитываем частоту появления каждого уникального символа
    return probability  # возвращаем итоговый словарь

In [ ]:
def buildSegmentDict(probability: Dict[str, Fraction]) -> Dict[str, List[Fraction]]:
    # Функция для составления "рабочих" полуинтервалов для каждого символа, где длина полуинтервала = частоте появления символа
    segmentDict = {}  # словарь для составления полуинтервалов (ключ - символ, значение - список, где 0-ой элемент - левая граница, а 1-ый элемент - правая граница)
    left = Fraction(0, 1)  # левая граница
    for char in probability.keys():  # итерируемся по уникальным символам и составляем "рабочий" полуинтервал для каждого символа
        segmentDict[char] = [left, left + probability[char]]
        left = left + probability[char]
    return segmentDict  # возвращаем итоговый словарь

In [ ]:
def encode(text: str, segmentDict: Dict[str, List[Fraction]]) -> Fraction:  # Функция для кодирования текста в число
    l, r = 0, 1  # левая граница полуинтервала = 0, правая = 1
    for char in text:  # итерируемся по каждому символу текста и пересчитываем левую и правую границу полуинтервала
        newRight = l + (r - l) * segmentDict[char][1]
        newLeft = l + (r - l) * segmentDict[char][0]
        l = newLeft
        r = newRight
    return (l + r) / 2  # в результате на текст - любое число из полуинтервала [l, r]. Для лучшей точности возьмем середину

In [ ]:
def decode(segmentDict: Dict[str, List[Fraction]], code: Fraction) -> str:  # Функция для декодирования числа в текст
    ans = ""  # наш текст, который должен получиться, лежит в переменной ans
    while True:  # запускаем бесконечный цикл, из которого мы выйдем только тогда, когда встретим остановочный символ
        for char, segment in segmentDict.items():  # итерируемся по словарю с "рабочими" полуинтервалами
            if char == "ඞ":  # если встретили @, то возвращаем ответ
                return ans
            if segment[0] <= code < segment[1]:  # если код(число - которое получили в процессе кодирования текста) лежит в полуинтервале [l, r], то это нужный нам символ
                ans += char  # кладем символ в ответ
                code = (code - segment[0]) / (segment[1] - segment[0])  # пересчитываем код для дальнейшего декодирования
                break

In [ ]:
def solution():
    with open("in.txt", 'r', encoding='utf-8') as file:
        textToEncode = file.read()
    probabilityDict = countProbability(textToEncode)  # словарь частот встречаемых символов
    segmentDict = buildSegmentDict(probabilityDict)  # словарь с "рабочими" полуинтервалами
    encodedResult = encode(textToEncode, segmentDict)  # закодированный текст
    print("Encoded text:", encodedResult)  # вывод числа
    decodedResult = decode(segmentDict, encodedResult)  # декодированное число(текст)
    print(decodedResult)  # вывод текста

In [8]:
solution()

Encoded text: 934973932447759254019170661893558357922003954531836936312161261992232719167753508537069896076816804329144646823711742155673863212239645664693959304758353607666814319864374492820691618487570358751566539951683945208720879640314904489214963593281566649192011058368546810438018818012659274118140870780371190427766155315259523133109233553062046750759146084106163043036092161582314504037488182593239594750833468118011653044526946125626611582269072659322160721144406588402261524702390569980301/26521499060013726569597671373379046683733575573704049827947620715911029840232870154635323349480916678523576344619495209491143811704856260215933651886690604040178487350554322259124965230375319512890325055737944497298432845407025464265247090453571290164068811931408314661580582283479472146145553579624275636519269032504905641290037469899189516058270570453029862191943877780505117900352314507301495630393941363681958612404278046351683044810971692560784639970806098248868302907984636133029444811
Все сим